In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import Dataset,DataLoader, TensorDataset 
from sklearn.utils import shuffle 
import matplotlib. pyplot as plt

In [2]:
def getSeq(start ,n ):
    x = [6*x+7 for x in range(start, start+n)]
    return x
    

In [3]:
data = []
for i in range(100):
    rnd = np.random.randint(0,25)
    data.append(getSeq(rnd,6))

data = np.array(data)
data = torch.from_numpy(data)

target = data[:,-1:].type(torch.FloatTensor)
data = data[:,:-1].type(torch.FloatTensor)

tarin_x = data[:90]
tarin_y = data[:90]
test_x = data[90:]
test_y = data[90:]

train_dataset = TensorDataset(tarin_x,tarin_y)
test_dataset = TensorDataset(test_x,test_y)
for i in test_dataset:
    print(i)
print(test_dataset)
train_loader=DataLoader(dataset=train_dataset, batch_size=5, shuffle=True) #训练数据加载器
test_loader=DataLoader(dataset=test_dataset, batch_size=5, shuffle=True)#测试数据加载器


(tensor([151., 157., 163., 169., 175.]), tensor([151., 157., 163., 169., 175.]))
(tensor([19., 25., 31., 37., 43.]), tensor([19., 25., 31., 37., 43.]))
(tensor([151., 157., 163., 169., 175.]), tensor([151., 157., 163., 169., 175.]))
(tensor([139., 145., 151., 157., 163.]), tensor([139., 145., 151., 157., 163.]))
(tensor([ 7., 13., 19., 25., 31.]), tensor([ 7., 13., 19., 25., 31.]))
(tensor([25., 31., 37., 43., 49.]), tensor([25., 31., 37., 43., 49.]))
(tensor([73., 79., 85., 91., 97.]), tensor([73., 79., 85., 91., 97.]))
(tensor([ 7., 13., 19., 25., 31.]), tensor([ 7., 13., 19., 25., 31.]))
(tensor([61., 67., 73., 79., 85.]), tensor([61., 67., 73., 79., 85.]))
(tensor([115., 121., 127., 133., 139.]), tensor([115., 121., 127., 133., 139.]))


In [4]:
#==========构建网络========== 
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(1,10,batch_first=True)
        self.lstm = nn.LSTM(1,10,1,batch_first = True)
        self.gru = nn.GRU(1,10,batch_first = True)
        self.fc = nn.Linear(10,1)
    def forward(self,x,hidden,c0): 
        output,(hidden,c0) = self.lstm(x,(hidden,c0))
        output = output[:,-1,:]
        output = self.fc(output)
        return output
        

In [5]:
net = model()
loss_fn = nn.MSELoss()
opt = torch.optim.Adam(net.parameters(),lr = 0.001)

In [6]:
h0 = torch.zeros(1,5,10)
c0 = torch.zeros(1,5,10)

for epoch in range(1200):
    for i,data in enumerate(train_loader):
        x,y = data 
        x=x.view(-1,5,1)

        pred = net(x,h0,c0)
        loss = loss_fn(pred,y)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
    
    if(epoch%100==0):
        print(loss.data)



D:\Mis\Anaconda\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([5, 5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(7824.1792)
tensor(10142.6709)
tensor(8056.4995)
tensor(6470.7246)
tensor(2218.9814)
tensor(876.7162)
tensor(853.7678)
tensor(203.4917)
tensor(371.1649)
tensor(92.6639)
tensor(72.9682)
tensor(73.8712)


In [7]:
rights = 0
length = 0
for i,data  in enumerate(test_loader):
    x,y =data
    x=x.view(-1,5,1)
    hidden = torch.zeros(1,5,10)
    pred = net(x,h0,c0)
    print(y.view(1,-1))
    print(pred.view(1,-1).data)


tensor([[  7.,  13.,  19.,  25.,  31., 151., 157., 163., 169., 175.,   7.,  13.,
          19.,  25.,  31., 151., 157., 163., 169., 175., 139., 145., 151., 157.,
         163.]])
tensor([[ 19.0681, 157.5084,  19.0681, 157.5084, 153.5356]])
tensor([[ 61.,  67.,  73.,  79.,  85.,  73.,  79.,  85.,  91.,  97.,  25.,  31.,
          37.,  43.,  49.,  19.,  25.,  31.,  37.,  43., 115., 121., 127., 133.,
         139.]])
tensor([[ 72.3741,  85.4759,  36.8813,  31.0547, 129.0752]])
